# IT24101629_Data_Cleaning

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
%matplotlib inline
sns.set(color_codes=True)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
BASE_PATH = '/content/drive/MyDrive/StudentStressLevelMonitoring'
RAW_XLSX = os.path.join(BASE_PATH, 'data/raw/StressLevelDataset1.xlsx')

In [5]:
OUTPUTS_DRIVE = os.path.join(BASE_PATH, 'results/outputs')
os.makedirs(OUTPUTS_DRIVE, exist_ok=True)

In [6]:
OUTPUTS_DRIVE_EDA= os.path.join(BASE_PATH, 'results/eda_visualizations')
os.makedirs(OUTPUTS_DRIVE, exist_ok=True)

In [7]:
df = pd.read_excel(RAW_XLSX)

In [8]:
df_before = df.copy()

In [9]:
def plot_missingness(dataframe, title, save_path):

    miss_cnt = dataframe.isnull().sum().sort_values(ascending=False)
    miss_pct = (miss_cnt / len(dataframe) * 100).round(2)
    miss_df = miss_pct[miss_cnt > 0]  # only columns with missing data

    plt.figure(figsize=(10, max(3, 0.35 * max(1, len(miss_df)))))
    if len(miss_df) > 0:
        sns.barplot(x=miss_df.values, y=miss_df.index)
        plt.xlabel("Missing (%)")
        plt.ylabel("Columns")
        plt.title(title)
        plt.tight_layout()
    else:
        plt.text(0.5, 0.5, "No missing values", ha="center", va="center", fontsize=14)
        plt.axis("off")
        plt.title(title)

    plt.savefig(save_path, dpi=150, bbox_inches="tight")
    plt.close()
    print(f"Saved: {save_path}")

In [10]:
print("BEFORE CLEANING")
print("Shape:", df_before.shape)
print("Missing values per column:\n", df_before.isnull().sum())

plot_missingness(
    df_before,
    "Missingness (Before Cleaning)",
    os.path.join(OUTPUTS_DRIVE_EDA, "missing_before.png")
)

df_before.to_csv(os.path.join(OUTPUTS_DRIVE, "raw_before_cleaning.csv"), index=False)


BEFORE CLEANING
Shape: (1119, 21)
Missing values per column:
 anxiety_level                   0
self_esteem                     5
mental_health_history           7
depression                      4
headache                        6
blood_pressure                  6
sleep_quality                   6
breathing_problem               6
noise_level                     4
living_conditions               7
safety                          7
basic_needs                     7
academic_performance            7
study_load                      7
teacher_student_relationship    7
future_career_concerns          6
social_support                  6
peer_pressure                   7
extracurricular_activities      4
bullying                        8
stress_level                    5
dtype: int64
Saved: /content/drive/MyDrive/StudentStressLevelMonitoring/results/eda_visualizations/missing_before.png


In [11]:
rows_before = len(df_before)
cols_before = df_before.shape[1]

#Drop missing values rows
df_clean = df_before.dropna(axis=0)

#Count duplicates and drop them
dup_count_before = df_clean.duplicated().sum()
df_clean = df_clean.drop_duplicates()

In [13]:
print("\n=== AFTER CLEANING ===")
print("Shape:", df_clean.shape)
print("Missing values per column:\n", df_clean.isnull().sum())

plot_missingness(
    df_clean,
    "Missingness (After Cleaning)",
    os.path.join(OUTPUTS_DRIVE_EDA, "missing_after.png")
)


rows_after = len(df_clean)
cols_after = df_clean.shape[1]


=== AFTER CLEANING ===
Shape: (1100, 21)
Missing values per column:
 anxiety_level                   0
self_esteem                     0
mental_health_history           0
depression                      0
headache                        0
blood_pressure                  0
sleep_quality                   0
breathing_problem               0
noise_level                     0
living_conditions               0
safety                          0
basic_needs                     0
academic_performance            0
study_load                      0
teacher_student_relationship    0
future_career_concerns          0
social_support                  0
peer_pressure                   0
extracurricular_activities      0
bullying                        0
stress_level                    0
dtype: int64
Saved: /content/drive/MyDrive/StudentStressLevelMonitoring/results/eda_visualizations/missing_after.png


In [14]:
dropped_rows = rows_before - rows_after
pct_dropped = (dropped_rows / rows_before * 100) if rows_before else 0

report = f"""
CLEANING REPORT
---------------
Rows before: {rows_before}
Rows after : {rows_after}
Dropped rows: {dropped_rows} ({pct_dropped:.2f}%)

Columns before: {cols_before}
Columns after : {cols_after}

Duplicates removed (pre-drop): {dup_count_before}
"""

print(report)

# Save report text
with open(os.path.join(OUTPUTS_DRIVE, "cleaning_report.txt"), "w") as f:
    f.write(report)

print(f"Saved: {os.path.join(OUTPUTS_DRIVE, 'cleaning_report.txt')}")


CLEANING REPORT
---------------
Rows before: 1119
Rows after : 1100
Dropped rows: 19 (1.70%)

Columns before: 21
Columns after : 21

Duplicates removed (pre-drop): 11

Saved: /content/drive/MyDrive/StudentStressLevelMonitoring/results/outputs/cleaning_report.txt


In [15]:
df_before.dropna().to_csv(os.path.join(OUTPUTS_DRIVE, "cleaned_dropna.csv"), index=False)

df_clean.to_csv(os.path.join(OUTPUTS_DRIVE, "cleaned_dropna_no_duplicates.csv"), index=False)

print("Saved cleaned CSVs to:")
print(" -", os.path.join(OUTPUTS_DRIVE, "cleaned_dropna.csv"))
print(" -", os.path.join(OUTPUTS_DRIVE, "cleaned_dropna_no_duplicates.csv"))

Saved cleaned CSVs to:
 - /content/drive/MyDrive/StudentStressLevelMonitoring/results/outputs/cleaned_dropna.csv
 - /content/drive/MyDrive/StudentStressLevelMonitoring/results/outputs/cleaned_dropna_no_duplicates.csv


In [16]:
TARGET = "stress_level"  # change if your target uses a different column name
if TARGET in df_before.columns:
    print("\nTarget balance (BEFORE):")
    print(df_before[TARGET].value_counts(dropna=False))
    print("\nTarget balance (AFTER):")
    print(df_clean[TARGET].value_counts(dropna=False))



Target balance (BEFORE):
stress_level
0.0    382
2.0    374
1.0    358
NaN      5
Name: count, dtype: int64

Target balance (AFTER):
stress_level
0.0    373
2.0    369
1.0    358
Name: count, dtype: int64
